In [1]:
import model_loader
import pipeline
from PIL import Image
from pathlib import Path
from transformers import CLIPTokenizer
import torch

DEVICE = "gpu"

ALLOW_CUDA = True
ALLOW_MPS = True

if torch.cuda.is_available() and ALLOW_CUDA:
    DEVICE = "cuda"
elif (torch.has_mps or torch.backends.mps.is_available()) and ALLOW_MPS:
    DEVICE = "mps"
print(f"Using device: {DEVICE}")

tokenizer = CLIPTokenizer("../data/vocab.json", merges_file="../data/merges.txt")
model_file = "../data/v1-5-pruned-emaonly.ckpt"
models = model_loader.preload_models_from_standard_weights(model_file, DEVICE)

## TEXT TO IMAGE

prompt = "sexy anime girl"
uncond_prompt = ""  
input_image = None
strength = 0.9

## SAMPLER

sampler = "ddpm"
num_inference_steps = 50
seed = 0

# Use torch.no_grad() to avoid memory issues with gradient calculation
with torch.no_grad():
    output_image = pipeline.generate(
        prompt=prompt,
        uncond_prompt=uncond_prompt,
        input_image=input_image,
        strength=strength,
        do_cfg=True,
        cfg_scale=8,
        sampler_name=sampler,
        n_inference_steps=num_inference_steps,
        seed=seed,
        models=models,
        device=DEVICE,
        idle_device="cpu",
        tokenizer=tokenizer,
    )

# Mostrar o guardar la imagen de salida
output_image_pil = Image.fromarray(output_image)
output_image_pil.show()  # Para mostrar la imagen
# output_image_pil.save("output_image.png")  # Para guardar la imagen en disco


Using device: cuda


100%|██████████| 50/50 [06:23<00:00,  7.68s/it]
